In [1]:
import matplotlib.pyplot as plt
import numpy as np
import subprocess
import plumed
import nglview
from ase.optimize.sciopt import *               
from ase.utils.geometry import *
from ase.lattice.spacegroup import crystal
from ase.visualize import *
from ase.lattice.surface import surface
from ase import Atoms
from ase import io
from ase.io import *
from ase.io.cif import read_cif
from ase.io.vasp import write_vasp
from abtem.visualize import show_atoms
from ase.visualize.plot import plot_atoms
from ase.build import add_adsorbate
from ase.io.proteindatabank import read_proteindatabank
from ase.io.lammpsdata import write_lammps_data

/home/ahlawat/.local/lib/python3.9/site-packages/ase/utils/geometry.py:8: UserWarning: Moved to ase.geometry and ase.build
  warnings.warn('Moved to ase.geometry and ase.build')
/home/ahlawat/.local/lib/python3.9/site-packages/ase/lattice/spacegroup.py:5: UserWarning: Moved to ase.spacegroup
  warnings.warn('Moved to ase.spacegroup')
/home/ahlawat/.local/lib/python3.9/site-packages/ase/lattice/surface.py:17: UserWarning: Moved to ase.build
  warnings.warn('Moved to ase.build')


In [ ]:
! rm *.vasp

## cubic 3C polymorph: http://dx.doi.org/10.1016/j.jpcs.2008.05.007
- Experimental data
   - ICSD 161481
   - ICSD Structure : Cs I3 Pb
   - Space Group: P m 3 m (221)
   - Cell: a 6.2894Å b 6.2894Å c 6.2894Å, α 90° β 90° γ 90° 

In [ ]:
structure = io.read('cubic.cif')
structure = sort(structure)
structure = structure.repeat((2,1,2))
structure = sort(structure)
view(structure)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20,10))

plot_atoms(structure, ax1, radii=0.5, rotation=('90x,0y,0z'))
#ax1.set_xlim(-20, 30)
ax1.set_xlabel(r'x[$\AA$]')
ax1.set_ylabel(r'z[$\AA$]')

plot_atoms(structure, ax2, radii=0.5, rotation=('90x,0y,0z'))
#ax2.set_xlim(-20, 30)
ax2.set_xlabel(r'x[$\AA$]')
ax2.set_ylabel(r'z[$\AA$]')

plot_atoms(structure, ax3, radii=0.5, rotation=('180x,0y,0z'))
#ax3.set_xlim(-20, 30)
ax3.set_xlabel(r'x[$\AA$]')
ax3.set_ylabel(r'y[$\AA$]')

#fig.savefig("ase_slab.png")
structure

## prepare vasp/quantum espresso input file

In [ ]:
write_vasp('cubic.vasp', structure, vasp5=True, direct=True)

inp_data={'prefix':"cubic",
          'electron_maxstep':1000,
          'outdir':"./",    
          'pseudo_dir':"/work/e89/e89/ahlawat/PP/",
          'calculation':'vc-relax',
          'vdw_corr':'grimme-d3',
          'ecutwfc':70.,
          'occupations':'fixed',
          'ecutrho':700.,   
          'conv_thr':1e-8,
          'electron_maxstep':500,
          'ion_dynamics':'bfgs',
          'cell_dynamics':'bfgs'}

pseudos={"Cs":"Cs.pbe-spn-rrkjus_psl.1.0.0.UPF",       
         "I":"I.pbe-n-rrkjus_psl.0.2.UPF", 
         "Pb":"Pb.pbe-dn-rrkjus_psl.0.2.2.UPF"}

io.write("cubic_cspbi3.pwi", 
         structure, 
         input_data=inp_data,
         pseudopotentials=pseudos, 
         kpts=(2, 4, 2),
         koffset=(0, 0, 0), 
         crystal_coordinates=False)


# k-points vs energy

In [ ]:
# final_e = [ -378.36256991, -378.09603049,  -378.16345971,  -378.14942638, -378.15382279, -378.15239590] 

# k_points = [1,2,3,4,5,6]

# fig, (ax) = plt.subplots(figsize=(6,6))

# ax.plot(k_points,final_e, color='magenta', ls='--', lw=2, markersize=10, marker='h')
# ax.set_xlabel('K-points', fontsize = 14)
# ax.set_ylabel('Total energy (Ry)', fontsize = 14)
# ax.tick_params(labelsize=12)
# ax.set_title('K-points vs Total energy, wfc=70 Ry', fontsize=15)

## wavefunction_cutoff tests  
  - improvement need to remove '' in numbers

In [ ]:
# structure = io.read('cubic.cif')
# structure = sort(structure)
# view(structure)
# import os
# newpath = r'./wfc_cutoff_test' 
# if not os.path.exists(newpath):
#     os.makedirs(newpath)
#     os.chdir(newpath)
    
# wf_cutoff = np.arange(20,140,10)

# for wfc in wf_cutoff:
#     rho_cut = wfc*10
#     inp_data={'prefix':"cubic-%d" %wfc, 
#               'electron_maxstep':1000,
#               'outdir':"./",    
#               'pseudo_dir':"/work/e89/e89/ahlawat/PP/",
#               'calculation':'relax',
#               'vdw_corr':'grimme-d3',
#               'ecutwfc': "%d" %wfc,
#               'occupations':'fixed',
#               'ecutrho':"%d" %rho_cut, 
#               'conv_thr':1e-8,
#               'electron_maxstep':500,
#               'ion_dynamics':'bfgs',
#               'cell_dynamics':'bfgs'}

#     pseudos={"Cs":"Cs.pbe-spn-rrkjus_psl.1.0.0.UPF",       
#               "I":"I.pbe-n-rrkjus_psl.0.2.UPF", 
#               "Pb":"Pb.pbe-dn-rrkjus_psl.0.2.2.UPF"}

#     io.write("cubic_cspbi3_%d.pwi" %wfc,
#               structure, 
#               input_data=inp_data,
#               pseudopotentials=pseudos, 
#               kpts=(2, 2, 2),
#               koffset=(0, 0, 0), 
#               crystal_coordinates=False)
# os.chdir("..")

In [ ]:
# final_e = [-378.06226432, -378.13984078, -378.14754140, -378.14812932, -378.14844286, -378.14925605,
#            -378.14942638, -378.14955884, -378.14969021, -378.14971276, -378.14974328, -378.14978518] 

# wfc = [ 20,  30,  40,  45,  50,  60,  70,  80,  90, 100, 110, 120]

# fig, (ax) = plt.subplots(figsize=(6,6))

# ax.plot(wfc,final_e, color='navy', ls='--', lw=2, markersize=10, marker='h')
# ax.set_xlabel('Wavefunction cutoff (Ry)', fontsize = 14)
# ax.set_ylabel('Total energy (Ry)', fontsize = 14)
# ax.tick_params(labelsize=12)
# ax.set_title('Wavefunction cutoff vs Total energy', fontsize=15)

## tetragonal polymorph:  https://dx.doi.org/10.25505/fiz.icsd.cc20vqj4
- Experimental data
  - Space Group: P 4/m b m (127)
  - Cell: a 8.82690(3)Å b 8.82690(3)Å c 6.29902(3)Å, α 90° β 90° γ 90° 

In [ ]:
structure = io.read('beta.cif')
structure = sort(structure)
structure = structure.repeat((1,1,2))
structure = sort(structure)

view(structure)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20,10))

plot_atoms(structure, ax1, radii=0.5, rotation=('90x,0y,0z'))
#ax1.set_xlim(-20, 30)
ax1.set_xlabel(r'x[$\AA$]')
ax1.set_ylabel(r'z[$\AA$]')

plot_atoms(structure, ax2, radii=0.5, rotation=('90x,0y,0z'))
#ax2.set_xlim(-20, 30)
ax2.set_xlabel(r'x[$\AA$]')
ax2.set_ylabel(r'z[$\AA$]')

plot_atoms(structure, ax3, radii=0.5, rotation=('180x,0y,0z'))
#ax3.set_xlim(-20, 30)
ax3.set_xlabel(r'x[$\AA$]')
ax3.set_ylabel(r'y[$\AA$]')

#fig.savefig("ase_slab.png")
structure

### prepare vasp/quantum espresso input file

In [ ]:
write_vasp('tetragonal.vasp', structure, vasp5=True, direct=True)

inp_data={'prefix':"tetragonal",
          'electron_maxstep':1000,
          'outdir':"./",    
          'pseudo_dir':"/work/e89/e89/ahlawat/PP/",
          'calculation':'vc-relax',
          'vdw_corr':'grimme-d3',
          'ecutwfc':70.,
          'occupations':'fixed',
          'ecutrho':700.,   
          'conv_thr':1e-8,
          'electron_maxstep':500,
          'ion_dynamics':'bfgs',
          'cell_dynamics':'bfgs'}

pseudos={"Cs":"Cs.pbe-spn-rrkjus_psl.1.0.0.UPF",       
         "I":"I.pbe-n-rrkjus_psl.0.2.UPF", 
         "Pb":"Pb.pbe-dn-rrkjus_psl.0.2.2.UPF"}

io.write("tetragonal_cspbi3.pwi", 
         structure, 
         input_data=inp_data,
         pseudopotentials=pseudos, 
         kpts=(3, 3, 2),
         koffset=(0, 0, 0), 
         crystal_coordinates=False)

## orthorhombic polymorph: http://dx.doi.org/10.1021/acsenergylett.8b00672
- Experimental data
  - ICSD 264725 
  - ICSD Structure : (Cs I3 Pb)n
  - Space Group: P n a m (62)
  - Cell: a 8.8561(4)Å b 8.5766(3)Å c 12.4722(6)Å, α 90° β 90° γ 90° 

In [ ]:
structure = io.read('ortho.cif')
structure = sort(structure)

view(structure)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20,10))

plot_atoms(structure, ax1, radii=0.5, rotation=('90x,0y,0z'))
#ax1.set_xlim(-20, 30)
ax1.set_xlabel(r'x[$\AA$]')
ax1.set_ylabel(r'z[$\AA$]')

plot_atoms(structure, ax2, radii=0.5, rotation=('90x,0y,0z'))
#ax2.set_xlim(-20, 30)
ax2.set_xlabel(r'x[$\AA$]')
ax2.set_ylabel(r'z[$\AA$]')

plot_atoms(structure, ax3, radii=0.5, rotation=('180x,0y,0z'))
#ax3.set_xlim(-20, 30)
ax3.set_xlabel(r'x[$\AA$]')
ax3.set_ylabel(r'y[$\AA$]')

#fig.savefig("ase_slab.png")
structure

### prepare vasp/quantum espresso input file

In [ ]:
write_vasp('ortho.vasp', structure, vasp5=True, direct=True)

inp_data={'prefix':"orthorhombic",
          'electron_maxstep':1000,
          'outdir':"./",    
          'pseudo_dir':"/work/e89/e89/ahlawat/PP/",
          'calculation':'vc-relax',
          'vdw_corr':'grimme-d3',
          'ecutwfc':70.,
          'occupations':'fixed',
          'ecutrho':700.,   
          'conv_thr':1e-8,
          'electron_maxstep':500,
          'ion_dynamics':'bfgs',
          'cell_dynamics':'bfgs'}

pseudos={"Cs":"Cs.pbe-spn-rrkjus_psl.1.0.0.UPF",       
         "I":"I.pbe-n-rrkjus_psl.0.2.UPF", 
         "Pb":"Pb.pbe-dn-rrkjus_psl.0.2.2.UPF"}

io.write("ortho_cspbi3.pwi", 
         structure, 
         input_data=inp_data,
         pseudopotentials=pseudos, 
         kpts=(3, 3, 2),
         koffset=(0, 0, 0), 
         crystal_coordinates=False)




## hexagonal polymorph: https://dx.doi.org/10.5517/cc11hdmk
- Experimental data
  - Crystal details
  - Space group 	P n m a (62)
  - Unit cell 	a 10.4342(7)Å b 4.7905(3)Å c 17.7610(10)Å
  - α 90.00° β 90.00° γ 90.00°
  - Cell volume 	887.78
  - Reduced cell 	a 4.791Å b 10.434Å c 17.761Å
  - α 90.000° β 90.000° γ 90.000°
  - Polymorph 	delta polymorph
  - Colour 	yellow


In [ ]:
structure = io.read('delta.cif')
structure = sort(structure)

view(structure)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20,10))

plot_atoms(structure, ax1, radii=0.5, rotation=('90x,0y,0z'))
#ax1.set_xlim(-20, 30)
ax1.set_xlabel(r'x[$\AA$]')
ax1.set_ylabel(r'z[$\AA$]')

plot_atoms(structure, ax2, radii=0.5, rotation=('90x,0y,0z'))
#ax2.set_xlim(-20, 30)
ax2.set_xlabel(r'x[$\AA$]')
ax2.set_ylabel(r'z[$\AA$]')

plot_atoms(structure, ax3, radii=0.5, rotation=('180x,0y,0z'))
#ax3.set_xlim(-20, 30)
ax3.set_xlabel(r'x[$\AA$]')
ax3.set_ylabel(r'y[$\AA$]')

structure
#fig.savefig("ase_slab.png")

### prepare vasp/quantum espresso input file

In [ ]:
write_vasp('delta.vasp', structure, vasp5=True, direct=True)

inp_data={'prefix':"hexagonal",
          'electron_maxstep':1000,
          'outdir':"./",    
          'pseudo_dir':"/work/e89/e89/ahlawat/PP/",
          'calculation':'vc-relax',
          'vdw_corr':'grimme-d3',
          'ecutwfc':70.,
          'occupations':'fixed',
          'ecutrho':700.,   
          'conv_thr':1e-8,
          'electron_maxstep':500,
          'ion_dynamics':'bfgs',
          'cell_dynamics':'bfgs'}

pseudos={"Cs":"Cs.pbe-spn-rrkjus_psl.1.0.0.UPF",       
         "I":"I.pbe-n-rrkjus_psl.0.2.UPF", 
         "Pb":"Pb.pbe-dn-rrkjus_psl.0.2.2.UPF"}

io.write("hexagonal_cspbi3.pwi", 
         structure, 
         input_data=inp_data,
         pseudopotentials=pseudos, 
         kpts=(2, 6, 1),
         koffset=(0, 0, 0), 
         crystal_coordinates=False)



# relative energies of different polymorphs of 20 atoms cells

In [ ]:
# # Cubic , tetragona, orthorhombic, hexagonal

# #Energies from DFT:
# PBE_D3 = [-1512.59911762, -1512.61689881, -1512.62056567, -1512.64799559] # in Ry
# PBE    = [-1512.25523334, -1512.28699951, -1512.29903323, -1512.31788605] # in Ry
# optB   = [-14.01453121, -14.59628122, -14.869, -15.48199443]              # in eV
# r2scan = [-643.62615739, -644.037, -644.19462648, -644.51920211]          # in eV

# #Energies converted to eV/f.u.:
# ar = np.array(opt)
# n = (ar + 14.01453121)*1000/4


# force fields simulations:
  - force fields are taken from : Bischak, Connor G., et al. "Liquid-like interfaces mediate structural phase transitions in lead halide perovskites." Matter 3.2 (2020): 534-545.

# cubic-phase

In [2]:
unit_cell = io.read('cubic.cif')
rep1 = 2
rep2 = 1
rep3 = 2

supercell = unit_cell.repeat((rep1,rep2,rep3))
supercell = sort(supercell)

# set point charges
i = 0
num_atoms = len(supercell.get_chemical_symbols())
charge_array = [0]*num_atoms
while i < num_atoms:
    if(supercell.get_chemical_symbols()[i] == 'Cs'):
        charge_array[i]= 0.86
    if(supercell.get_chemical_symbols()[i] == 'I'):
        charge_array[i]= -0.57
    if(supercell.get_chemical_symbols()[i] == 'Pb'):
        charge_array[i]= 0.85
    i = i + 1

supercell.set_initial_charges(charges=charge_array)
view(supercell)
print(supercell) 


write_lammps_data('data.CPI', supercell, atom_style = 'full', force_skew=True, units='real')

with open("start.lmp","w") as f:
    print("""
###
dimension       3
boundary        p p p
units           metal
atom_style      full
variable        seed world 1428
variable        freq equal 500
read_data       data.CPI
mass            1 132.904999 # Cs
mass            2 126.900002 # I
mass            3 204.199997 # Pb
pair_style      lj/cut/coul/long 8.0 10.0
pair_modify     mix arithmetic
neigh_modify    every 1
kspace_style    pppm 1e-4
dielectric 1.0
variable ACs  equal "0.07728"
variable rCs  equal "3.584"
variable APb  equal "0.001086"
variable rPb  equal "3.210"
variable AI   equal "0.06389"
variable rI   equal "4.014"
pair_coeff      1 1 ${ACs}    ${rCs}
pair_coeff      2 2 ${AI}     ${rI}
pair_coeff      3 3 ${APb}    ${rPb}
thermo          ${freq}
thermo_style    custom step temp pe ke etotal press lx ly lz xy xz yz
# Minimization

min_style cg
fix 1 all box/relax aniso 0.0 vmax 0.01
minimize        1.0e-4 1.0e-6 1000 10000
unfix 1
write_data      data.min
""",file=f)

subprocess.run("lmp_mpi < start.lmp ",shell=True)

/home/ahlawat/.local/lib/python3.9/site-packages/ase/io/cif.py:401: UserWarning: crystal system 'cubic' is not interpreted for space group Spacegroup(221, setting=1). This may result in wrong setting!
  warnings.warn(


Atoms(symbols='Cs4I12Pb4', pbc=True, cell=[12.5788, 6.2894, 12.5788], initial_charges=..., spacegroup_kinds=...)
LAMMPS (23 Jun 2022)
Reading data file ...
  triclinic box = (0 0 0) to (12.5788 6.2894 12.5788) with tilt (0 0 0)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  20 atoms
Finding 1-2 1-3 1-4 neighbors ...
  special bond factors lj:    0        0        0       
  special bond factors coul:  0        0        0       
     0 = max # of 1-2 neighbors
     0 = max # of 1-3 neighbors
     0 = max # of 1-4 neighbors
     1 = max # of special neighbors
  special bonds CPU = 0.001 seconds
  read_data CPU = 0.003 seconds
PPPM initialization ...
  using 12-bit tables for long-range coulomb (../kspace.cpp:342)
  G vector (1/distance) = 0.27767082
  grid = 8 4 8
  stencil order = 5
  estimated absolute RMS force accuracy = 0.00039700966
  estimated relative force accuracy = 2.7570795e-05
  using double precision KISS FFT
  3d grid and FFT values/proc = 2475 256
Generated 3 of 3

CompletedProcess(args='lmp_mpi < start.lmp ', returncode=0)

# tetragonal-phase

In [3]:
unit_cell = io.read('beta.cif')
rep1 = 1
rep2 = 1
rep3 = 2

supercell = unit_cell.repeat((rep1,rep2,rep3))
supercell = sort(supercell)

# set point charges
i = 0
num_atoms = len(supercell.get_chemical_symbols())
charge_array = [0]*num_atoms
while i < num_atoms:
    if(supercell.get_chemical_symbols()[i] == 'Cs'):
        charge_array[i]= 0.86
    if(supercell.get_chemical_symbols()[i] == 'I'):
        charge_array[i]= -0.57
    if(supercell.get_chemical_symbols()[i] == 'Pb'):
        charge_array[i]= 0.85
    i = i + 1

supercell.set_initial_charges(charges=charge_array)
#view(supercell)
print(supercell) 


write_lammps_data('data.CPI', supercell, atom_style = 'full', force_skew=True, units='real')

with open("start.lmp","w") as f:
    print("""
###
dimension       3
boundary        p p p
units           metal
atom_style      full
variable        seed world 1428
variable        freq equal 500
read_data       data.CPI
mass            1 132.904999 # Cs
mass            2 126.900002 # I
mass            3 204.199997 # Pb
pair_style      lj/cut/coul/long 8.0 10.0
pair_modify     mix arithmetic
neigh_modify    every 1
kspace_style    pppm 1e-4
dielectric 1.0
variable ACs  equal "0.07728"
variable rCs  equal "3.584"
variable APb  equal "0.001086"
variable rPb  equal "3.210"
variable AI   equal "0.06389"
variable rI   equal "4.014"
pair_coeff      1 1 ${ACs}    ${rCs}
pair_coeff      2 2 ${AI}     ${rI}
pair_coeff      3 3 ${APb}    ${rPb}
thermo          ${freq}
thermo_style    custom step temp pe ke etotal press lx ly lz xy xz yz
# Minimization

min_style cg
fix 1 all box/relax aniso 0.0 vmax 0.01
minimize        1.0e-4 1.0e-6 1000 10000
unfix 1
write_data      data.min
""",file=f)

subprocess.run("lmp_mpi < start.lmp ",shell=True)

/home/ahlawat/.local/lib/python3.9/site-packages/ase/io/cif.py:401: UserWarning: crystal system 'tetragonal' is not interpreted for space group Spacegroup(127, setting=1). This may result in wrong setting!
  warnings.warn(


Atoms(symbols='Cs4I12Pb4', pbc=True, cell=[8.8269, 8.8269, 12.59804], initial_charges=..., spacegroup_kinds=...)
LAMMPS (23 Jun 2022)
Reading data file ...
  triclinic box = (0 0 0) to (8.8269 8.8269 12.59804) with tilt (0 0 0)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  20 atoms
Finding 1-2 1-3 1-4 neighbors ...
  special bond factors lj:    0        0        0       
  special bond factors coul:  0        0        0       
     0 = max # of 1-2 neighbors
     0 = max # of 1-3 neighbors
     0 = max # of 1-4 neighbors
     1 = max # of special neighbors
  special bonds CPU = 0.001 seconds
  read_data CPU = 0.003 seconds
PPPM initialization ...
  using 12-bit tables for long-range coulomb (../kspace.cpp:342)
  G vector (1/distance) = 0.28388955
  grid = 6 6 8
  stencil order = 5
  estimated absolute RMS force accuracy = 0.00027746631
  estimated relative force accuracy = 1.9268969e-05
  using double precision KISS FFT
  3d grid and FFT values/proc = 2535 288
Generated 3 of 3

Abort(1) on node 0 (rank 0 in comm 0): application called MPI_Abort(MPI_COMM_WORLD, 1) - process 0


CompletedProcess(args='lmp_mpi < start.lmp ', returncode=1)

# orthorhombic-phase

In [6]:
unit_cell = io.read('ortho.cif')
rep1 = 1
rep2 = 1
rep3 = 1

supercell = unit_cell.repeat((rep1,rep2,rep3))
supercell = sort(supercell)

# set point charges
i = 0
num_atoms = len(supercell.get_chemical_symbols())
charge_array = [0]*num_atoms
while i < num_atoms:
    if(supercell.get_chemical_symbols()[i] == 'Cs'):
        charge_array[i]= 0.86
    if(supercell.get_chemical_symbols()[i] == 'I'):
        charge_array[i]= -0.57
    if(supercell.get_chemical_symbols()[i] == 'Pb'):
        charge_array[i]= 0.85
    i = i + 1

supercell.set_initial_charges(charges=charge_array)
#view(supercell)
#print(supercell) 


write_lammps_data('data.CPI', supercell, atom_style = 'full', force_skew=True, units='real')

with open("start.lmp","w") as f:
    print("""
###
dimension       3
boundary        p p p
units           metal
atom_style      full
variable        seed world 1428
variable        freq equal 500
read_data       data.CPI
mass            1 132.904999 # Cs
mass            2 126.900002 # I
mass            3 204.199997 # Pb
pair_style      lj/cut/coul/long 8.0 10.0
pair_modify     mix arithmetic
neigh_modify    every 1
kspace_style    pppm 1e-4
dielectric 1.0
variable ACs  equal "0.07728"
variable rCs  equal "3.584"
variable APb  equal "0.001086"
variable rPb  equal "3.210"
variable AI   equal "0.06389"
variable rI   equal "4.014"
pair_coeff      1 1 ${ACs}    ${rCs}
pair_coeff      2 2 ${AI}     ${rI}
pair_coeff      3 3 ${APb}    ${rPb}
thermo          ${freq}
thermo_style    custom step temp pe ke etotal press lx ly lz xy xz yz
# Minimization

min_style cg
fix 1 all box/relax aniso 0.0 vmax 0.01
minimize        1.0e-4 1.0e-6 1000 10000
unfix 1
write_data      data.min
""",file=f)
    
# ## run MD
# variable        temperature equal 300.0
# variable        temperature2 equal 300.0
# variable        tempDamp equal 100.0 # approx 0.1 ps
# variable        pressure equal 1.00
# variable        pressureDamp equal 500.0
# # NVT
# dump            myDump all dcd 500 out.0.dcd
# fix             1 all temp/csvr ${temperature} ${temperature} ${tempDamp} ${seed}
# fix             2 all nve
# timestep        0.002
# velocity        all create ${temperature} ${seed} dist gaussian
# run             10000
# unfix           1
# unfix           2
# write_data      data.NVT
# # NPT
# fix             1 all temp/csvr ${temperature} ${temperature2} ${tempDamp} ${seed}
# fix             2 all nph tri ${pressure} ${pressure} ${pressureDamp}
# fix             3 all momentum 10000 linear 1 1 1
# run             10000
# unfix           1
# unfix           2
# unfix           3
# undump          myDump
# reset_timestep  0
# write_restart   restart.file
# write_data      data.eq

subprocess.run("lmp_mpi < start.lmp ",shell=True)

LAMMPS (23 Jun 2022)
Reading data file ...
  triclinic box = (0 0 0) to (8.856 8.576 12.472) with tilt (0 0 0)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  20 atoms
Finding 1-2 1-3 1-4 neighbors ...
  special bond factors lj:    0        0        0       
  special bond factors coul:  0        0        0       
     0 = max # of 1-2 neighbors
     0 = max # of 1-3 neighbors
     0 = max # of 1-4 neighbors
     1 = max # of special neighbors
  special bonds CPU = 0.001 seconds
  read_data CPU = 0.003 seconds
PPPM initialization ...
  using 12-bit tables for long-range coulomb (../kspace.cpp:342)
  G vector (1/distance) = 0.28471587
  grid = 6 6 8
  stencil order = 5
  estimated absolute RMS force accuracy = 0.00026881823
  estimated relative force accuracy = 1.8668393e-05
  using double precision KISS FFT
  3d grid and FFT values/proc = 2535 288
Generated 3 of 3 mixed pair_coeff terms from arithmetic mixing rule
Neighbor list info ...
  update every 1 steps, delay 0 steps, che

CompletedProcess(args='lmp_mpi < start.lmp ', returncode=0)

In [ ]:
traj = read('out.1.lammpstrj', index=":", parallel=True)
view(traj)

# hexagonal-phase

In [5]:
unit_cell = io.read('delta.cif')
rep1 = 1
rep2 = 1
rep3 = 1

supercell = unit_cell.repeat((rep1,rep2,rep3))
supercell = sort(supercell)

# set point charges
i = 0
num_atoms = len(supercell.get_chemical_symbols())
charge_array = [0]*num_atoms
while i < num_atoms:
    if(supercell.get_chemical_symbols()[i] == 'Cs'):
        charge_array[i]= 0.86
    if(supercell.get_chemical_symbols()[i] == 'I'):
        charge_array[i]= -0.57
    if(supercell.get_chemical_symbols()[i] == 'Pb'):
        charge_array[i]= 0.85
    i = i + 1

supercell.set_initial_charges(charges=charge_array)
view(supercell)
#print(supercell) 


write_lammps_data('data.CPI', supercell, atom_style = 'full', force_skew=True, units='real')

with open("start.lmp","w") as f:
    print("""
###
dimension       3
boundary        p p p
units           metal
atom_style      full
variable        seed world 1428
variable        freq equal 500
read_data       data.CPI
mass            1 132.904999 # Cs
mass            2 126.900002 # I
mass            3 204.199997 # Pb
pair_style      lj/cut/coul/long 8.0 10.0
pair_modify     mix arithmetic
neigh_modify    every 1
kspace_style    pppm 1e-4
dielectric 1.0
variable ACs  equal "0.07728"
variable rCs  equal "3.584"
variable APb  equal "0.001086"
variable rPb  equal "3.210"
variable AI   equal "0.06389"
variable rI   equal "4.014"
pair_coeff      1 1 ${ACs}    ${rCs}
pair_coeff      2 2 ${AI}     ${rI}
pair_coeff      3 3 ${APb}    ${rPb}
thermo          ${freq}
thermo_style    custom step temp pe ke etotal press lx ly lz xy xz yz
# Minimization

min_style cg
fix 1 all box/relax aniso 0.0 vmax 0.01
minimize        1.0e-4 1.0e-6 1000 10000
unfix 1
write_data      data.min
""",file=f)

subprocess.run("lmp_mpi < start.lmp ",shell=True)

LAMMPS (23 Jun 2022)
Reading data file ...
  triclinic box = (0 0 0) to (10.4342 4.7905 17.761) with tilt (0 0 0)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  20 atoms
Finding 1-2 1-3 1-4 neighbors ...
  special bond factors lj:    0        0        0       
  special bond factors coul:  0        0        0       
     0 = max # of 1-2 neighbors
     0 = max # of 1-3 neighbors
     0 = max # of 1-4 neighbors
     1 = max # of special neighbors
  special bonds CPU = 0.000 seconds
  read_data CPU = 0.003 seconds
PPPM initialization ...
  using 12-bit tables for long-range coulomb (../kspace.cpp:342)
  G vector (1/distance) = 0.2797659
  grid = 8 5 9
  stencil order = 5
  estimated absolute RMS force accuracy = 0.00037362396
  estimated relative force accuracy = 2.5946748e-05
  using double precision KISS FFT
  3d grid and FFT values/proc = 2880 360
Generated 3 of 3 mixed pair_coeff terms from arithmetic mixing rule
Neighbor list info ...
  update every 1 steps, delay 0 steps, c

CompletedProcess(args='lmp_mpi < start.lmp ', returncode=0)

In [ ]:
ar = np.array(scan)
n = (ar + 643.62615739)*1000/4
n

In [7]:
38.91 -38.698463  

0.21153699999999986

In [9]:
211/20

10.55